In [1]:
# !pip install cloud-tpu-client==0.10 torch==1.12.0 https://storage.googleapis.com/tpu-pytorch/wheels/colab/torch_xla-1.12-cp37-cp37m-linux_x86_64.whl
# import torch
# import torch_xla
# import torch_xla.core.xla_model as xm
# device = xm.xla_device()
# print(device)

In [2]:
import os
import random

class DataBuilder:
    def __init__(self, in_path, labels_72, labels_5):
        self.data = []
        for ldir in os.listdir(in_path):
            total = 0
            if "72" in ldir : limit = labels_72
            elif "5" in ldir : limit = labels_5
            for individuals in os.listdir(os.path.join(in_path, ldir)):
                if total<limit:
                    labels = os.listdir(os.path.join(in_path, ldir))
                    labels = list(set(labels)-{individuals})
                    random.shuffle(labels)
                    images = os.listdir(os.path.join(in_path, ldir, individuals))
                    anchor = "0.png"
                    images = list(set(images)-{anchor})
                    anchor = "/".join([in_path, ldir, individuals, anchor])
                    for i in range(len(images)):
                        pos = "/".join([in_path, ldir, individuals, images[i]])
                        if i != 0:
                            label = os.listdir(os.path.join(in_path, ldir, labels[i]))
                            random.shuffle(label)
                            label = label[0]
                            neg = "/".join([in_path, ldir, labels[i], label])
                        else:
                            label = os.listdir(os.path.join(in_path, ldir, labels[i]))[0]
                            neg = "/".join([in_path, ldir, labels[i], label])
                        self.data.append([anchor, pos, neg])
                    total += 1


path = "/kaggle/input/face-recognition"
# path = "/".join(path)
data = DataBuilder(path, 1000, 1000)

In [3]:
import random
data = data.data
random.shuffle(data)
for i in data[:5]:
    print(i)

['/kaggle/input/face-recognition/subjects_0-1999_72_imgs/503/0.png', '/kaggle/input/face-recognition/subjects_0-1999_72_imgs/503/35.png', '/kaggle/input/face-recognition/subjects_0-1999_72_imgs/962/28.png']
['/kaggle/input/face-recognition/subjects_0-1999_72_imgs/32/0.png', '/kaggle/input/face-recognition/subjects_0-1999_72_imgs/32/62.png', '/kaggle/input/face-recognition/subjects_0-1999_72_imgs/27/45.png']
['/kaggle/input/face-recognition/subjects_0-1999_72_imgs/857/0.png', '/kaggle/input/face-recognition/subjects_0-1999_72_imgs/857/60.png', '/kaggle/input/face-recognition/subjects_0-1999_72_imgs/228/3.png']
['/kaggle/input/face-recognition/subjects_0-1999_72_imgs/405/0.png', '/kaggle/input/face-recognition/subjects_0-1999_72_imgs/405/62.png', '/kaggle/input/face-recognition/subjects_0-1999_72_imgs/700/53.png']
['/kaggle/input/face-recognition/subjects_0-1999_72_imgs/442/0.png', '/kaggle/input/face-recognition/subjects_0-1999_72_imgs/442/53.png', '/kaggle/input/face-recognition/subjec

In [4]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision.io import read_image, ImageReadMode
from sklearn.model_selection import train_test_split

class FaceDataset(Dataset):
    def __init__(self, data):
        self.data = data
    
    def __getitem__(self, index):
        anchor, positive, negative = self.data[index]
        anchor = read_image(anchor, mode=ImageReadMode.RGB)
        positive = read_image(positive, mode=ImageReadMode.RGB)
        negative = read_image(negative, mode=ImageReadMode.RGB)
        return torch.as_tensor(anchor, dtype=torch.float32), torch.as_tensor(positive, dtype=torch.float32), torch.as_tensor(negative, dtype=torch.float32)
    
    def __len__(self):
        return len(self.data)

In [5]:
import torch
from torch import nn as nn
from torch import optim as optim
from torch.nn import functional as F
from torch.optim import lr_scheduler


class Model(nn.Module):
    def __init__(self, in_channels, embedding_size, p_dropout, p_linear_dropout):
        super(Model, self).__init__()
        self.conv = nn.Sequential(nn.Conv2d(in_channels, in_channels, kernel_size=1, padding=0),
                                   nn.AvgPool2d(kernel_size=2, stride=2))
        

        self.b1 = nn.Sequential(nn.Conv2d(in_channels, 64, kernel_size=1, padding="same"),
                                nn.BatchNorm2d(64),
                                nn.LeakyReLU(0.1))
        self.b2 = nn.Sequential(nn.Conv2d(in_channels, 92, kernel_size=1, padding="same"),
                                nn.BatchNorm2d(92),
                                nn.LeakyReLU(0.1),
                                nn.Conv2d(92, 128, kernel_size=3, padding="same"),
                                nn.BatchNorm2d(128),
                                nn.LeakyReLU(0.1),)
        self.b3 = nn.Sequential(nn.Conv2d(in_channels, 24, kernel_size=1, padding="same"),
                                nn.BatchNorm2d(24),
                                nn.LeakyReLU(0.1),
                                nn.Conv2d(24, 32, kernel_size=5, padding="same"),
                                nn.BatchNorm2d(32),
                                nn.LeakyReLU(0.1))
        self.b4 = nn.Sequential(nn.MaxPool2d(kernel_size=3, padding=1, stride = 1),
                                nn.Conv2d(in_channels, 32, kernel_size=1, padding="same"),
                                nn.BatchNorm2d(32),
                                nn.LeakyReLU(0.1))
        

        self.b5 = nn.Sequential(nn.Conv2d(256, 32, kernel_size=1, padding="same"),
                                nn.BatchNorm2d(32),
                                nn.LeakyReLU(0.1))
        self.b6 = nn.Sequential(nn.Conv2d(256, 128, kernel_size=1, padding="same"),
                                nn.BatchNorm2d(128),
                                nn.LeakyReLU(0.1),
                                nn.Conv2d(128, 64, kernel_size=3, padding="same"),
                                nn.BatchNorm2d(64),
                                nn.LeakyReLU(0.1),)
        self.b7 = nn.Sequential(nn.Conv2d(256, 24, kernel_size=1, padding="same"),
                                nn.BatchNorm2d(24),
                                nn.LeakyReLU(0.1),
                                nn.Conv2d(24, 16, kernel_size=5, padding="same"),
                                nn.BatchNorm2d(16),
                                nn.LeakyReLU(0.1))
        self.b8 = nn.Sequential(nn.MaxPool2d(kernel_size=3, padding=1, stride = 1),
                                nn.Conv2d(256, 16, kernel_size=1, padding="same"),
                                nn.BatchNorm2d(16),
                                nn.LeakyReLU(0.1))

        self.dropout = nn.Dropout2d(p_dropout)
        
        self.fc = nn.Sequential(nn.Linear(401408, 512, bias=True),
                                nn.BatchNorm1d(512),
                                nn.LeakyReLU(0.1),
                                nn.Dropout(p_linear_dropout),
                                nn.Linear(512, embedding_size, bias=True))
        
        
    def forward(self, x, batch_size):
        x = self.conv(x)

        xb1 = self.b1(x)
        xb2 = self.b2(x)
        xb3 = self.b3(x)
        xb4 = self.b4(x)
        x = torch.cat([xb1, xb2, xb3, xb4], dim = 1)

        x = self.dropout(x)

        xb5 = self.b5(x)
        xb6 = self.b6(x)
        xb7 = self.b7(x)
        xb8 = self.b8(x)
        x = torch.cat([xb5, xb6, xb7, xb8], dim = 1)

        x = self.dropout(x)

        
        x = x.view(batch_size, -1)
        x = self.fc(x)

        return x

class TripletLoss:
    def __init__(self, margin, p):
        self.margin = margin
        self.p = p

    def loss(self, anchor, positive, negative):
        distance_ap = F.pairwise_distance(anchor, positive, p = self.p)
        distance_an = F.pairwise_distance(anchor, negative, p = self.p)
        tripletloss = torch.clamp(input=distance_ap+(self.margin)-distance_an, min=0.0).mean()
        return tripletloss

class Optimizer:
    def __init__(self, model, lr):
        self.model = model
        self.lr = lr
    
    def optimize(self):
        return optim.Adam(self.model.parameters(), lr=self.lr)

class Schedular:
    def __init__(self, optimizer, lr, step_size, gamma):
        self.optimizer = optimizer
        self.lr = lr
        self.step_size = step_size
        self.gamma = gamma
    
    def decay(self):
        return lr_scheduler.StepLR(self.optimizer, self.step_size, self.gamma)

In [6]:
class Variables:
    def __init__(self, in_channels=3, embedding_size=256, batch_size=40, batch_size_test_val = 40, lr=0.001, margin=0.3, p=2, epochs=100, step_size=3, gamma=0.5, p_dropout=0.2, p_linear_dropout=0.2):
        self.batch_size = batch_size
        self.lr = lr
        self.in_channels = in_channels
        self.emembedding_size =embedding_size
        self.p = p
        self.margin = margin
        self.epochs = epochs
        self.step_size = step_size
        self.gamma = gamma
        self.batch_size_test_val = batch_size_test_val
        self.p_dropout = p_dropout
        self.p_linear_dropout = p_linear_dropout

In [ ]:

import pandas as pd
import numpy as np
import warnings
from tqdm import tqdm
import torch
from torch import nn as nn
from torch import optim as optim
from torch.nn import functional as F
from torch.utils.data import DataLoader


if __name__ == "__main__":
    # warnings.filterwarnings("ignore", category=UserWarning, module="torch.nn.init")

    train = data[0:70000]
    val = data[67000:72000]
    test = data[70000:75000]

    train = FaceDataset(train)
    val = FaceDataset(val)
    test = FaceDataset(test)
    variables = Variables()
    train = DataLoader(train, batch_size = variables.batch_size, shuffle=False)
    val = DataLoader(val, batch_size = variables.batch_size_test_val, shuffle=False)
    test = DataLoader(test, batch_size = variables.batch_size_test_val, shuffle=False)

    print("train:", len(train))
    print("val:", len(val))
    print("test:", len(test))
    
    model = Model(in_channels=variables.in_channels, embedding_size=variables.emembedding_size, p_dropout=variables.p_dropout, p_linear_dropout=variables.p_linear_dropout)
    tripletloss = TripletLoss(margin=variables.margin, p=variables.p)
    optimizer = Optimizer(model=model, lr=variables.lr).optimize()
    schedular = Schedular(optimizer, variables.lr, variables.step_size, variables.gamma).decay()
    device = "cuda"
    # device = xm.xla_device()
    model.to(device)

    for epoch in range(0, variables.epochs):
        model.train()
        train_cost = 0.0
        loop = tqdm(enumerate(train), total=len(train), leave=True)
        for _, train_batch in loop:
            anc, pos, neg = train_batch
            anc, pos, neg = anc.to(device), pos.to(device), neg.to(device)
            anc = model.forward(anc, variables.batch_size)
            pos = model.forward(pos, variables.batch_size)
            neg = model.forward(neg, variables.batch_size)
            loss = tripletloss.loss(anc, pos, neg)
            train_cost += loss
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loop.set_description(f"[{epoch+1}/{variables.epochs}]")
            loop.set_postfix(loss=loss.item())
        avg_train_loss = train_cost/len(train)
        
        model.eval()
        with torch.no_grad():
            val_cost = 0.0
            loop_val = tqdm(enumerate(val), total=len(val), leave=True)
            for _, val_batch in loop_val:
                anc, pos, neg = val_batch
                anc, pos, neg = anc.to(device), pos.to(device), neg.to(device)
                anc = model.forward(anc, variables.batch_size_test_val)
                pos = model.forward(pos, variables.batch_size_test_val)
                neg = model.forward(neg, variables.batch_size_test_val)
                val_loss = tripletloss.loss(anc, pos, neg)
                val_cost += val_loss.item()
                loop_val.set_description(f"[{epoch+1}/{variables.epochs}]")
                loop_val.set_postfix(loss=val_loss.item())
            avg_val_loss = val_cost/len(val)
            print(f"\nEpoch {epoch+1}/{variables.epochs}, Training Loss: {avg_train_loss}, Validation Loss: {avg_val_loss}")
        checkpoint_filename = f"model_epoch_{epoch+1}.pth"
        if avg_train_loss < 1:
            if avg_val_loss < 1:
                torch.save(model.state_dict(), checkpoint_filename)
        elif (epoch+1)%5==0:
            torch.save(model.state_dict(), checkpoint_filename)
        schedular.step()
        print("\n")


train: 1750
val: 125
test: 125


[1/100]: 100%|██████████| 125/125 [00:41<00:00,  3.03it/s, loss=0.402]



Epoch 1/100, Training Loss: 0.40805330872535706, Validation Loss: 0.2013580079227686




[2/100]: 100%|██████████| 125/125 [00:40<00:00,  3.09it/s, loss=0.207] 



Epoch 2/100, Training Loss: 0.22244086861610413, Validation Loss: 0.09741024102270603




[3/100]:  83%|████████▎ | 1455/1750 [11:44<02:19,  2.11it/s, loss=0.0916] 

In [ ]:
torch.save(model.state_dict(), "abcd.pth")

In [ ]:
import torch
import torch_xla
import torch_xla.core.xla_model as xm
xm.rendezvous('clear_memory')  # This can help clear TPU memory
torch.cuda.empty_cache()